In [217]:
%matplotlib inline

import colorsys
import random
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
from bokeh.plotting import figure, show
from bokeh.resources import CDN
from bokeh.io import output_notebook
output_notebook( resources=CDN )
import math

pd.set_option('max_colwidth', 200)

Loading BokehJS ...

In [218]:
# Tube
lines       = pd.read_csv('london.lines.csv', index_col=0)
stations    = pd.read_csv('london.stations.csv', index_col=0)
stationsRein= pd.read_csv('london.stations.reindexed.csv', index_col=0)
connections = pd.read_csv('london.connections.csv')

# Bikes
bikesRawData       = pd.read_csv('bikes/london-bikes-coords.csv', index_col=0)

In [219]:
tubeGraph = nx.Graph()

for connection_id, connection in connections.iterrows():
    station1_name = stations.ix[connection['station1']]['name']
    station2_name = stations.ix[connection['station2']]['name']
    tubeGraph.add_edge(station1_name, station2_name, time = connection['time'])
    
#add the connection between Bank and Monument manually
tubeGraph.add_edge('Bank', 'Monument', time = 1)
# nx.draw(tubeGraph)

/usr/local/Cellar/ipython/6.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
/usr/local/Cellar/ipython/6.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [220]:
nx.shortest_path(tubeGraph, 'Oxford Circus', 'Canary Wharf', weight='time')

['Oxford Circus',
 'Tottenham Court Road',
 'Holborn',
 'Chancery Lane',
 "St. Paul's",
 'Bank',
 'Shadwell',
 'Wapping',
 'Rotherhithe',
 'Canada Water',
 'Canary Wharf']

In [221]:
# Print Bike Data
bikesRawData

,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,EndLatitude,EndLongitude,StartLatitude,StartLongitude
Rental Id,,,,,,,,,,,,
73667188,720,7315,6/3/18 7:18,282,"Royal London Hospital, Whitechapel",6/3/18 7:06,30,"Windsor Terrace, Hoxton",51.518222,-0.058518,51.529239,-0.093476
73693268,540,3500,6/3/18 23:14,95,"Aldersgate Street, Barbican",6/3/18 23:05,751,"Newton Street, Covent Garden",51.520091,-0.097608,51.516573,-0.121343
73627763,1560,6139,3/3/18 20:32,60,"Lisson Grove, St. John's Wood",3/3/18 20:06,203,"West Smithfield Rotunda, Farringdon",51.524779,-0.168315,51.518248,-0.100737
73638919,480,12516,4/3/18 20:04,107,"Finsbury Leisure Centre, St. Luke's",4/3/18 19:56,203,"West Smithfield Rotunda, Farringdon",51.526022,-0.095231,51.518248,-0.100737
73623157,1320,7203,3/3/18 12:56,496,"Devons Road, Bow",3/3/18 12:34,446,"York Hall, Bethnal Green",51.522163,-0.018298,51.530140,-0.055168
73607211,540,3959,1/3/18 14:29,521,"Driffield Road, Old Ford",1/3/18 14:20,446,"York Hall, Bethnal Green",51.533171,-0.036762,51.530140,-0.055168
73592503,900,14948,28/02/2018 07:25,508,"Fournier Street, Whitechapel",28/02/2018 07:10,70,"Calshot Street , King's Cross",51.519277,-0.073497,51.532791,-0.117255
73675110,300,14663,6/3/18 9:34,136,"Queen Victoria Street, St. Paul's",6/3/18 9:29,95,"Aldersgate Street, Barbican",51.511713,-0.099315,51.520091,-0.097608
73678145,120,7194,6/3/18 12:29,339,"Risinghill Street, Angel",6/3/18 12:27,234,"Liverpool Road (N1 Centre), Angel",51.533199,-0.112322,51.534534,-0.106988


In [222]:
normed = stations[['longitude', 'latitude']]
normed = normed - normed.min()
normed = normed / normed.max()
locations = dict(zip(stations['name'], normed[['longitude', 'latitude']].values))

p = figure(
    x_range = (.4,.7),
    y_range = (.2,.5),
    height= 700,
    width= 900,
)
for edge in tubeGraph.edges():
    p.line( 
        x= [locations[pt][0] for pt in edge],
        y= [locations[pt][1] for pt in edge],
    )

for node in tubeGraph.nodes():
    x = [locations[node][0]]
    y = [locations[node][1]]
    p.circle(
        x, y, 
        line_alpha=0)
    p.text(
        x, y, 
        text = {'value':node}, 
        text_font_size = str(10) +"pt",
        text_align='center',
        text_font_style='bold')
    
show(p)

In [223]:
nx.algorithms.connectivity.cuts.minimum_node_cut(tubeGraph)

{'Warwick Avenue'}

In [224]:
nx.algorithms.connectivity.stoerwagner.stoer_wagner(tubeGraph)

(1,
 (['Upminster'],
  ['Chiswick Park',
   'Stamford Brook',
   'Bond Street',
   'West India Quay',
   'Northwick Park',
   'Arnos Grove',
   'Highbury & Islington',
   'Island Gardens',
   'South Ruislip',
   'Canons Park',
   'Wimbledon Park',
   'West Hampstead',
   'Camden Town',
   'Stockwell',
   'Hounslow West',
   'Woodside Park',
   'Moorgate',
   'North Acton',
   'Aldgate',
   'Beckton',
   'Elverson Road',
   'Putney Bridge',
   'New Cross Gate',
   'Cockfosters',
   'Bromley-By-Bow',
   'South Woodford',
   'Bounds Green',
   'Richmond',
   'Upminster Bridge',
   'Colindale',
   'Highgate',
   'West Ham',
   'Morden',
   'Southfields',
   'Tottenham Court Road',
   'Brixton',
   'Warren Street',
   'Beckton Park',
   "King's Cross St. Pancras",
   'Upton Park',
   'South Kensington',
   'Alperton',
   'Wanstead',
   'Becontree',
   'Plaistow',
   'Canary Wharf',
   'Upney',
   'Hounslow East',
   "Shepherd's Bush (C)",
   'Hampstead',
   'Fairlop',
   'Chalk Farm',
   'R

In [225]:
# BIKES
uniqueBikeLocations = {}
uniqueBikeCount = 0

for id, info in bikesRawData.iterrows():
    if info["StartStation Name"] not in uniqueBikeLocations:
        name = info["StartStation Name"]
        lat = info["StartLatitude"]
        long = info["StartLongitude"]
        uniqueBikeLocations[name] = (lat, long)
        uniqueBikeCount += 1
        
    if info["EndStation Name"] not in uniqueBikeLocations:
        name = info["EndStation Name"]
        lat = info["EndLatitude"]
        long = info["EndLongitude"]
        uniqueBikeLocations[name] = (lat, long)
        uniqueBikeCount += 1
        
print("UniqueBikeCount: ",uniqueBikeCount)

UniqueBikeCount:  733


In [226]:
# list of unique bike locations ("stations") and their coordinates
pd.DataFrame.from_dict(uniqueBikeLocations, orient="index", columns=['latitude', 'longitude'])

,latitude,longitude
"Windsor Terrace, Hoxton",51.529239,-0.093476
"Royal London Hospital, Whitechapel",51.518222,-0.058518
"Newton Street, Covent Garden",51.516573,-0.121343
"Aldersgate Street, Barbican",51.520091,-0.097608
"West Smithfield Rotunda, Farringdon",51.518248,-0.100737
"Lisson Grove, St. John's Wood",51.524779,-0.168315
"Finsbury Leisure Centre, St. Luke's",51.526022,-0.095231
"York Hall, Bethnal Green",51.530140,-0.055168
"Devons Road, Bow",51.522163,-0.018298
"Driffield Road, Old Ford",51.533171,-0.036762


In [304]:
### Get Random value and Mean


# Calcuates mean coordinates based on Tube Stations
latcoors = stationsRein['latitude']
longcoors = stationsRein['longitude']
latmean = latcoors.mean()
longmean = longcoors.mean()

# meancoors = (latmean, longmean)

# Center of London
meancoors = (51.510776, -0.115638)

print("Mean Coordinates: ", meancoors)
longitudeBoundSize = 0.2 # Mess with these for bigger radius
latitudeBoundSize = 0.2  # Mess with these for bigger radius

randLat1 = meancoors[0] + (random.uniform(0,1)*latitudeBoundSize - (latitudeBoundSize/2))
randLong1 = meancoors[1] + (random.uniform(0,1)*longitudeBoundSize - (longitudeBoundSize/2))
randLat2 = meancoors[0] + (random.uniform(0,1)*latitudeBoundSize - (latitudeBoundSize/2))
randLong2 = meancoors[1] + (random.uniform(0,1)*longitudeBoundSize - (longitudeBoundSize/2))

print("Random origin location:        ", randLat1,"," ,randLong1)
print("Random destination location:   ", randLat2,",", randLong2)

Mean Coordinates:  (51.510776, -0.115638)
Random origin location:         51.52585489012914 , -0.12116914225065124
Random destination location:    51.51278901862392 , -0.08213810734390019


In [305]:
# Calculates the closest TUBE stations to the random coordinates

#calculate distances to all stations in array from co-ordinates
tubeDistances1 = []
tubeDistances2 = []
for i in range(1, len(stationsRein['longitude'])):
    distance1 = math.sqrt(((randLat1 - stationsRein['latitude'].iloc[i-1])*(11/7))**2 + (randLong1 - stationsRein['longitude'].iloc[i-1])**2)
    distance2 = math.sqrt(((randLat2 - stationsRein['latitude'].iloc[i-1])*(11/7))**2 + (randLong2 - stationsRein['longitude'].iloc[i-1])**2)
    #print(stationsRein['name'].iloc[i-1])
    #print(i)
    #print(stationsRein['latitude'].iloc[i-1])
    #print(distance1)
    
    tubeDistances1.append(distance1)
    tubeDistances2.append(distance2)

#pick shorest for each co-ordinate
station1 = tubeDistances1.index(min(tubeDistances1))
station2 = tubeDistances2.index(min(tubeDistances2))
#print(station1)
#print(distances1[station1])
print("Nearest station to Random Origin:", stationsRein['name'][station1+1], stationsRein['latitude'][station1+1], stationsRein['longitude'][station1+1])
print("Nearest station to Random Destination:", stationsRein['name'][station2+1],stationsRein['latitude'][station2+1], stationsRein['longitude'][station2+1])



Nearest station to Random Origin: Russell Square 51.523 -0.1244
Nearest station to Random Destination: Monument 51.5108 -0.0863


In [306]:
# Calculate the closest BIKE station to the random coordinates

# Calculate distances to all stations in array from co-ordinates
shortestBikeOriginName = ""
shortestBikeDestinationName = ""
shortestBikeOriginValue = 9999
shortestBikeDestinationValue = 9999

for bikeLoc in uniqueBikeLocations:
    bName = bikeLoc
    bLat  = uniqueBikeLocations[bikeLoc][0]
    bLong = uniqueBikeLocations[bikeLoc][1]
    distanceFromRandomToOrigin      = math.sqrt(((randLat1 - bLat)*11/7)**2 + (randLong1 - bLong)**2)
    distanceFromRandomToDestination = math.sqrt(((randLat2 - bLat)*11/7)**2 + (randLong2 - bLong)**2)
    
    if distanceFromRandomToOrigin < shortestBikeOriginValue:
        shortestBikeOriginName = bName
        shortestBikeOriginValue = distanceFromRandomToOrigin
        
    if distanceFromRandomToDestination < shortestBikeDestinationValue:
        shortestBikeDestinationName = bName
        shortestBikeDestinationValue = distanceFromRandomToDestination


print("Nearest bike station to Random Origin:", shortestBikeOriginName, uniqueBikeLocations[shortestBikeOriginName])
print("Nearest bike station to Random Destination:", shortestBikeDestinationName, uniqueBikeLocations[shortestBikeDestinationName])

Nearest bike station to Random Origin: Brunswick Square, Bloomsbury (51.5242289, -0.12260789999999999)
Nearest bike station to Random Destination: St. Mary Axe, Aldgate (51.51367570000001, -0.07858)


In [307]:
bikesRawData

,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,EndLatitude,EndLongitude,StartLatitude,StartLongitude
Rental Id,,,,,,,,,,,,
73667188,720,7315,6/3/18 7:18,282,"Royal London Hospital, Whitechapel",6/3/18 7:06,30,"Windsor Terrace, Hoxton",51.518222,-0.058518,51.529239,-0.093476
73693268,540,3500,6/3/18 23:14,95,"Aldersgate Street, Barbican",6/3/18 23:05,751,"Newton Street, Covent Garden",51.520091,-0.097608,51.516573,-0.121343
73627763,1560,6139,3/3/18 20:32,60,"Lisson Grove, St. John's Wood",3/3/18 20:06,203,"West Smithfield Rotunda, Farringdon",51.524779,-0.168315,51.518248,-0.100737
73638919,480,12516,4/3/18 20:04,107,"Finsbury Leisure Centre, St. Luke's",4/3/18 19:56,203,"West Smithfield Rotunda, Farringdon",51.526022,-0.095231,51.518248,-0.100737
73623157,1320,7203,3/3/18 12:56,496,"Devons Road, Bow",3/3/18 12:34,446,"York Hall, Bethnal Green",51.522163,-0.018298,51.530140,-0.055168
73607211,540,3959,1/3/18 14:29,521,"Driffield Road, Old Ford",1/3/18 14:20,446,"York Hall, Bethnal Green",51.533171,-0.036762,51.530140,-0.055168
73592503,900,14948,28/02/2018 07:25,508,"Fournier Street, Whitechapel",28/02/2018 07:10,70,"Calshot Street , King's Cross",51.519277,-0.073497,51.532791,-0.117255
73675110,300,14663,6/3/18 9:34,136,"Queen Victoria Street, St. Paul's",6/3/18 9:29,95,"Aldersgate Street, Barbican",51.511713,-0.099315,51.520091,-0.097608
73678145,120,7194,6/3/18 12:29,339,"Risinghill Street, Angel",6/3/18 12:27,234,"Liverpool Road (N1 Centre), Angel",51.533199,-0.112322,51.534534,-0.106988


In [313]:
# Calculate BIKE travel time
# Calculates the mean of the closest bike ports, as well as ones that are really close.

exactMatches = 0
relativeMatches = 0
totalDurationAllTrips = 0
for entry_id, entry in bikesRawData.iterrows():
    eStartName =  entry["StartStation Name"]
    eStartCoords = (entry["StartLatitude"], entry["StartLongitude"])
    
    eEndName   =  entry["EndStation Name"]
    eEndCoords = (entry["EndLatitude"], entry["EndLongitude"])
    
    eDuration = entry["Duration"]
    
    if shortestBikeOriginName == eStartName and shortestBikeDestinationName == eEndName :
        exactMatches += 1
        totalDurationAllTrips += eDuration
        continue
        
    #   Also include trips from really nearby stations..
    startLatEp = abs(eStartCoords[0] - uniqueBikeLocations[shortestBikeOriginName][0]) < 0.02
    startLongEp = abs(eStartCoords[1] - uniqueBikeLocations[shortestBikeOriginName][1]) < 0.02
    endLatEp = abs(eEndCoords[0] - uniqueBikeLocations[shortestBikeDestinationName][0]) < 0.02
    endLatEp = abs(eEndCoords[1] - uniqueBikeLocations[shortestBikeDestinationName][1]) < 0.02
    #  print(startLatEp, startLongEp, endLatEp, endLatEp)
    if startLatEp and startLongEp and endLatEp and endLatEp:
        relativeMatches += 1
        totalDurationAllTrips += eDuration
    
print(exactMatches)
print(relativeMatches)
print(totalDurationAllTrips / (exactMatches + relativeMatches))
print("The average time of this BIKE trip is:", (totalDurationAllTrips / (exactMatches + relativeMatches))/60, "minutes.")



0
3339
1042.8032345013478
The average time of this BIKE trip is: 17.380053908355798 minutes.


In [312]:
# Calculate TRAINN travel time
# Calculates with dijkstra's shortest path algorithm (weighted by TIME)

closestOriginStationName = stationsRein['name'][station1+1]
closestDestStationName = stationsRein['name'][station2+1]

nx.algorithms.shortest_paths.dijkstra_path(tubeGraph, closestOriginStationName, closestDestStationName)

quickestTrainTime = nx.algorithms.shortest_paths.dijkstra_path_length(tubeGraph,closestOriginStationName,closestDestStationName, weight = "time")

print("The quickest time for this TRAIN trip is ",quickestTrainTime, "minutes.")



The quickest time for this TRAIN trip is  8 minutes.
